数据集：https://github.com/fatecbf/toutiao-text-classfication-dataset/blob/master/toutiao_cat_data.txt.zip

搜索引擎的三个步骤：
- 自动下载网页(网页爬虫)
- 建立索引
- 度量网页质量(pagerank)

In [3]:
data = open('./toutiao_cat_data.txt').readlines()

In [4]:
from collections import Counter
import pandas as pd
import jieba
import numpy as np

In [5]:
data[1]

'6552368441838272771_!_101_!_news_culture_!_发酵床的垫料种类有哪些？哪种更好？_!_\n'

In [6]:
data = [d.split('_!_') for d in data]

In [7]:
df = pd.DataFrame(data, columns=['id', 'code', 'category', 'content', 'keys'])

In [13]:
# 跑不动就减小数据集
# df = df[:50000]

In [56]:
NUM_DOCUMENTS = len(df)
df.head()

,id,code,category,content,keys
0,6551700932705387022,101,news_culture,京城 最 值得 你 来场 文化 之旅 的 博物馆,"保利集团,马未都,中国科学技术馆,博物馆,新中国\n"
1,6552368441838272771,101,news_culture,发酵 床 的 垫料 种类 有 哪些 ？ 哪 种 更好 ？,\n
2,6552407965343678723,101,news_culture,上联 ： 黄山 黄河 黄皮肤 黄土高原 。 怎么 对 下联 ？,\n
3,6552332417753940238,101,news_culture,林徽因 什么 理由 拒绝 了 徐志摩 而 选择 梁思成 为 终身伴侣 ？,\n
4,6552475601595269390,101,news_culture,黄杨木 是 什么 树 ？,\n


# 进行分词

In [15]:
df['content'] = df.content.apply(lambda x: ' '.join(jieba.cut(x)))
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.831 seconds.
Prefix dict has been built succesfully.


,id,code,category,content,keys
0,6551700932705387022,101,news_culture,京城 最 值得 你 来场 文化 之旅 的 博物馆,"保利集团,马未都,中国科学技术馆,博物馆,新中国\n"
1,6552368441838272771,101,news_culture,发酵 床 的 垫料 种类 有 哪些 ？ 哪 种 更好 ？,\n
2,6552407965343678723,101,news_culture,上联 ： 黄山 黄河 黄皮肤 黄土高原 。 怎么 对 下联 ？,\n
3,6552332417753940238,101,news_culture,林徽因 什么 理由 拒绝 了 徐志摩 而 选择 梁思成 为 终身伴侣 ？,\n
4,6552475601595269390,101,news_culture,黄杨木 是 什么 树 ？,\n


In [16]:
# 创建字典
vocab = []
for document in df.content:
    vocab.extend(document.split())

# 总单词数目
NUM_TREMS = len(vocab)

vocab = dict(Counter(vocab))
# 根据词频排序
vocab = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
# 转换成字典
vocab = dict(vocab)
# 过滤少于5000词频的单词
vocab = {k:v for k,v in vocab.items() if v>100}
# word2id id2word
word2id = {k:v for v,k in enumerate(vocab)}
id2word = {v:k for k,v in word2id.items()}
# 字典数量
NUM_WORDS = len(vocab)
NUM_WORDS, # vocab.keys(), word2id

(693,)

# 创建索引

最简单的索引就是对每一个文档(网页)创建一个很长的二进制数字，每个数字表示其对应的关键字是否在文档当中。

``` 
way1

         文档1  文档2 ...
关键字1    1      0
关键字2    0      1
....
```

In [18]:
def generate_index(x):
    """
    将字符串转换成id list
    """
    index = [0] * NUM_WORDS
    words = x.split()
    for word in words:
        i = word2id.get(word, -1)
        if i == -1: continue
        index[i] = 1
    return index


# 验证一下是否是想要的结果
r = generate_index('京城 最 值得 你 来场 文化 之旅 的 博物馆')
# [id2word[int(w)] for w in """35
# 203
# 7
# 394
# 2002
# 2
# 2180""".split()]

> ps: 需要运行很长时间，占用很大内存条

In [23]:
all_index = df.content.apply(generate_index)

``` 
way2
       关键字1  关键字2 关键字3 ...
文档1    1        0       0
文档2    0        1       1
...
```
将其转置一下就可以了

In [24]:
all_index = np.array(all_index.tolist()).T
all_index.shape

(693, 50000)

这样会生成一个表，即索引表：表的每一行对应一个关键字，而每一个关键字后面跟着一组数字，表示哪些文档包含这个关键字。  
这时候，当我们查询某个关键字的时候，不再需要将整个文档集合从头到尾查看一遍，只需要查找这个表就知道应该取哪些文档中去搜索就可以了。  
这和我们生活中的书籍的目录是非常相似的。  

当我们想要查询多个关键字的文档时候，也只需要将两个关键字对应的索引向量进行一个布尔运算(and)，便能将所有出现两个关键字的文档检索出来。  
计算机对于布尔运算的计算速度是非常快的。

假如互联网有$100亿^3$($10^{10}$)个有用的网页，再假设词汇表有30万个单词(肯定偏小)，那么索引的大小至少是100亿x30万=3000万亿。  
假如我们去掉一些只出现一个网页(文档)中的词，取百分之一，那也有30万亿。  

不仅这样，索引除了保存关键字信息，还要有一些其他信息需要保存，比如词出现的位置啦，次数啦，  
这些数据一台服务器是村放不下的，所以通常把索引切片，放到不同服务器上，这就和服务器的集群，分布式很相关了。

将数据存到不同的服务器上，就会引申出很多其它问题，比如，如何保证数据的一致性(数据脏读等等)，如何降低服务期间的通信成本，否则分布式效率还不如一台服务器效果高。

总之，由搜索引擎的索引可以扩展到很多很深的知识，这些知识偏向工程化，但工程化的问题不管多么复杂，至少在索引的原理上，还是非常简单的。类似目录，等价于布尔运算。

# 度量网页质量

Google的pagerank方法是常用的网页质量度量方法，在NLP领域中也是比较重要的一个方法。由于此数据集不适用于pagerank方法，这里就不再展开。  
仅做简单介绍，有兴趣的同学可自行深入理解研究，或在开课吧NLP课程中进行学习。

在掌握
- 数据下载
- 建立索引
- 度量网页

后，我们再来看下搜索引擎的其它考量：
1. 索引要完善，巧妇难为无米之炊，算法再好，要找的答案不再候选答案集合中，那都是徒劳的。
2. 网页质量的度量，一些八卦性质的网站pagerank值比较高，但权威性质就比较低
3. 用户偏好，不同的用户对结果的评判标准不一样，个性化推荐(深入就是走向推荐的方向了)
4. 确定查询和网页的相关性

OK，我们来看怎么确定网页和查询的相关性的度量

# 确定网页和查询的相关性

我们常用词频(Term Frequency)的概念来表示：  
$$tf = \frac{单词的出现频率}{网页的总字数}$$

所以，一个文档中的单词出现的词频可以表示为：$TF_1, TF_2, ...$  
那么假如给定关键字$w_1, w_2$，可以计算每个文档中两个关键字的词频，记为$TF_1, TF_2$  

比如 `北京` `首都`   在文档 `可爱 的 北京 是 祖国 的 首都， 北京 是 个 美丽 的 城市` 和 文档`他 喊道 北京 北京 我爱你， 北京 北京 我 来了， 首都 北京 我 来 了`
词频分别是
[2/13, 1/13]
[5/14, 1/14]

所以，我们可以把查询关键字和文档的相关性的度量表示为：
$TF_1+TF_2$

当有N个关键字，M个文档时候，第i个文档的相似度可以记为：

$$sim(q, d_i) = TF_1+TF_2+....TF_N, (q=w_1,w_2,...w_n)$$

In [124]:
# 北京的房价多少
query = '北京的房价多少？'
#query = '特朗普是什么的？'
query = list(jieba.cut(query))

# 过滤不再字典的
query = [word for word in query if word in vocab]
query

['北京', '的', '房价', '多少', '？']

计算每个关键字的TF

In [125]:
def querybykeys(query):
    finds = np.array([1] * NUM_DOCUMENTS)
    for key in query:
        finds += all_index[word2id[key]]
        finds = finds==2
        finds = finds.astype(int)
    return finds

finds = querybykeys(query)
finds = [print(i) for i,j in enumerate(finds) if j]
print('Find document id that contains all keys:', finds)

Find document id that contains all keys: []


In [126]:
# 既然没有那咱找几个小的数据进行继续的说明
finds = querybykeys(['北京', '房价']) # ['北京', '房价']
finds = [i for i,j in enumerate(finds) if j]
print('Find document id that contains all keys:', finds)

# 将这些数据打印出来
for d in finds:
    print(df.iloc[d].content)

Find document id that contains all keys: [11848, 21857, 23181, 23516, 25454, 28720, 29154, 45713]
房价 白跌 了 ！ 北京 首 套房 贷款 利率 上浮 10%
你 认为 北京 未来 5 年 的 房价 会 下跌 吗 ？
「 重磅 」 北京 限 房价 项目 销售 办法 出炉   部分 将 收作 共有 产权 房
北京 的 房价 还会 继续 上涨 吗 ？
北京 房贷利率 下周 再涨   专家 ： 累计 相当于 房价 上调 了 10%
北京 限价 房转 共有 产权 房 政策 拟 出炉   对 房价 有何 影响 ？
房价 不涨 卖 不动 ， 北京 首 套房 利率 继续 上浮 ， 刚需 招惹 了 谁 ？
北京 的 房价 还会 继续 上涨 吗 ？


In [123]:
# 计算每个数据的TF
tf_result = []
for d in finds:
    words = df.iloc[d].content.split()
    # 统计单词出现次数
    tf = dict(Counter(words))
    # 统计词频
    tf = {k:v/len(tf) for k,v in tf.items() if k in query}
    # 统计query的词频
    print(words)
    print(tf)
    print(sum(tf.values()))
    tf_result.append(list(tf.values()))
    
tf_result

['房价', '白跌', '了', '！', '北京', '首', '套房', '贷款', '利率', '上浮', '10%']
{'房价': 0.09090909090909091, '北京': 0.09090909090909091}
0.18181818181818182
['你', '认为', '北京', '未来', '5', '年', '的', '房价', '会', '下跌', '吗', '？']
{'北京': 0.08333333333333333, '的': 0.08333333333333333, '房价': 0.08333333333333333}
0.25
['「', '重磅', '」', '北京', '限', '房价', '项目', '销售', '办法', '出炉', '部分', '将', '收作', '共有', '产权', '房']
{'北京': 0.0625, '房价': 0.0625}
0.125
['北京', '的', '房价', '还会', '继续', '上涨', '吗', '？']
{'北京': 0.125, '的': 0.125, '房价': 0.125}
0.375
['北京', '房贷利率', '下周', '再涨', '专家', '：', '累计', '相当于', '房价', '上调', '了', '10%']
{'北京': 0.08333333333333333, '房价': 0.08333333333333333}
0.16666666666666666
['北京', '限价', '房转', '共有', '产权', '房', '政策', '拟', '出炉', '对', '房价', '有何', '影响', '？']
{'北京': 0.07142857142857142, '房价': 0.07142857142857142}
0.14285714285714285
['房价', '不涨', '卖', '不动', '，', '北京', '首', '套房', '利率', '继续', '上浮', '，', '刚需', '招惹', '了', '谁', '？']
{'房价': 0.0625, '北京': 0.0625}
0.125
['北京', '的', '房价', '还会', '继续', '上涨', '吗', '？']
{'北京': 

[[0.09090909090909091, 0.09090909090909091],
 [0.08333333333333333, 0.08333333333333333, 0.08333333333333333],
 [0.0625, 0.0625],
 [0.125, 0.125, 0.125],
 [0.08333333333333333, 0.08333333333333333],
 [0.07142857142857142, 0.07142857142857142],
 [0.0625, 0.0625],
 [0.125, 0.125, 0.125]]

这里面大家有没有发现一个问题？  
因为数据量比较少，可能不太明显，  
但你还是能够感受得到的。

在sim最大的两条数据中，`北京` `房价` `的` 三个单词分别贡献了0.125的权重  
但从直观理解来说，`的`字对于问题其实完全没有帮助

**TF-IDF**

In [ ]:
tf = dict(Counter(vocab))
tf = {k:round(v/words,8) for k,v in tf.items()}
tf = sorted(tf.items(), key=lambda x:x[1], reverse=True)
# tf